In [1]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)
    
with open("config_tests.yaml", "r") as stream:
    config_tests = yaml.safe_load(stream)
    
N_EPOCHS = 50
N_TRIALS = 10
BATCH_SIZE = 512

CONFIG = config_tests["proba_best"]

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [3]:
ds_1 = tfds.load("stl10", split="train", as_supervised=True, shuffle_files=False)
ds_2 = tfds.load("stl10", split="test", as_supervised=True, shuffle_files=False)

x = []
y = []
for x_, y_ in ds_1.as_numpy_iterator():
    x.append(x_)
    y.append(y_)
for x_, y_ in ds_2.as_numpy_iterator():
    x.append(x_)
    y.append(y_)
    
x = np.array(x, dtype=np.float32) / 255
y = np.array(y, dtype=np.float32)

2024-04-28 08:07:02.218319: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-04-28 08:07:02.970800: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=42, stratify=y)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42, stratify=y_test)

n_classes = len(np.unique(y))
train_size = x_train.shape[0]
train_size, len(x_val), len(x_test)

(7800, 2600, 2600)

In [5]:
x_train_sorted = x_train[np.argsort(y_train)]
y_train_sorted = y_train[np.argsort(y_train)]
_, counts = np.unique(y_train_sorted, return_counts=True)

In [11]:
model = ClassifierModel(output_shape=n_classes, **models_hyperparameters["test_model"])
model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
model(x_train[0:1])
# model.save_weights("../models/default_model.weights.h5")
model.load_weights("../models/default_model.weights.h5")
model_weights = model.get_weights()

In [16]:
model_scores = []
verbose = 1
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=5, start_from_epoch=0)

for _ in tqdm(range(N_TRIALS)):
    model.set_weights(model_weights)
    
    for j in range(len(model.layers[:-1])):
        model.layers[j].layers[-1].rate = 0.25
    
    for i in range(N_EPOCHS):
        n_samples = int(np.tanh(2 * (i + 1) / N_EPOCHS) * train_size)
        
        samples_proba = utils.calculate_proba(
            model, x_train_sorted, y_train_sorted, counts
        )
        
        samples_ids = utils.chose_samples(n_samples, samples_proba, CONFIG["order_type"])
        model.fit(
            x_train_sorted[samples_ids],
            y_train_sorted[samples_ids],
            # validation_data=(x_val, y_val),
            epochs=1,
            batch_size=BATCH_SIZE,
            verbose=verbose,
        )
        
        for j in range(len(model.layers[:-1])):
            model.layers[j].layers[-1].rate -= 0.003
    
    model.fit(
        x_train, y_train, validation_data=(x_val, y_val), epochs=500, batch_size=BATCH_SIZE, 
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=5, start_from_epoch=0)], 
        verbose=verbose
    )
            
    _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=verbose)
    model_scores.append(accuracy)
    print("Mean:", np.mean(model_scores), " Median: ", np.median(model_scores))

  0%|          | 0/10 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.1158 - loss: 2.5107
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.1547 - loss: 2.4123
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step - accuracy: 0.2051 - loss: 2.2274
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 0.1827 - loss: 2.2050
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.2247 - loss: 2.1524
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.2414 - loss: 2.0771
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.2230 - loss: 2.1076
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.2534 - loss: 2.0117
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.2720 - loss: 1.9700
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.2773 - loss: 1.9417
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.3034 - loss: 1.8911
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.3232 - loss: 1.8453
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.3197 - loss: 1.8351
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.3287 - los

 10%|█         | 1/10 [02:18<20:46, 138.45s/it]

Mean: 0.5615384578704834  Median:  0.5615384578704834
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 387ms/step - accuracy: 0.1029 - loss: 2.5544
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 350ms/step - accuracy: 0.1082 - loss: 2.4760
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step - accuracy: 0.1405 - loss: 2.3833
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 234ms/step - accuracy: 0.1780 - loss: 2.2589
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 157ms/step - accuracy: 0.2003 - loss: 2.1742
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 149ms/step - accuracy: 0.2233 - loss: 2.1732
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 123ms/step - accuracy: 0.2058 - loss: 2.1648
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - accuracy: 0.1946 - loss: 2.1404
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 122ms/step - accuracy: 0.2439 - loss: 2.0500
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - accuracy: 0.2522 - loss: 2.0104
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 118ms/step - accuracy: 0.2817 - loss: 1.9446
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - accuracy: 0.3303 - loss: 1.8667
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 103ms/step - accuracy: 0.3278 - loss: 1.83

 20%|██        | 2/10 [04:25<17:36, 132.02s/it]

...

Mean: 0.5416666666666666  Median:  0.5350000262260437
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step - accuracy: 0.1061 - loss: 2.5161
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 356ms/step - accuracy: 0.1056 - loss: 2.4526
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 428ms/step - accuracy: 0.1519 - loss: 2.3669
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 238ms/step - accuracy: 0.1797 - loss: 2.2480
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 152ms/step - accuracy: 0.2345 - loss: 2.1431
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - accuracy: 0.2328 - loss: 2.1121
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 147ms/step - accuracy: 0.2416 - loss: 2.0870
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 130ms/step - accuracy: 0.2401 - loss: 2.0532
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step - accuracy: 0.2680 - loss: 1.9873
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 113ms/step - accuracy: 0.2921 - loss: 1.9273
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step - accuracy: 0.2770 - loss: 1.9485
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - accuracy: 0.2913 - loss: 1.9172
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 102ms/step - accuracy: 0.3103 - loss: 1.86

 40%|████      | 4/10 [08:53<13:17, 132.95s/it]

Mean: 0.5407692342996597  Median:  0.5365384817123413
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step - accuracy: 0.1158 - loss: 2.5346
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 440ms/step - accuracy: 0.1110 - loss: 2.5004
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 426ms/step - accuracy: 0.1496 - loss: 2.4124
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 226ms/step - accuracy: 0.1619 - loss: 2.3038
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 155ms/step - accuracy: 0.1849 - loss: 2.2558
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 156ms/step - accuracy: 0.2145 - loss: 2.1411
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - accuracy: 0.2217 - loss: 2.0904
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step - accuracy: 0.2571 - loss: 2.0309
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 113ms/step - accuracy: 0.2700 - loss: 2.0247
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step - accuracy: 0.2851 - loss: 1.9867
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - accuracy: 0.2821 - loss: 1.9642
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step - accuracy: 0.3022 - loss: 1.9192
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - accuracy: 0.3109 - loss: 1.877

 40%|████      | 4/10 [09:48<14:43, 147.17s/it]


KeyboardInterrupt: 

In [17]:
model_scores

[0.5615384578704834,
 0.5284615159034729,
 0.5350000262260437,
 0.5380769371986389]

In [8]:
model.layers[1].layers[-1].rate

0.1359999999999999

In [41]:
model_scores_random = []
verbose = 0

for _ in tqdm(range(N_TRIALS)):
    model.set_weights(model_weights)

    for j in range(len(model.layers[:-1])):
        model.layers[j].layers[-1].rate = 0.3

    model.fit(
        x_train, y_train, validation_data=(x_val, y_val), epochs=500, batch_size=BATCH_SIZE, verbose=verbose, 
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=5, start_from_epoch=35)]
    )

    _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=verbose)
    model_scores_random.append(accuracy)
    print("Mean:", np.mean(model_scores_random), " Median: ", np.median(model_scores_random))

 10%|█         | 1/10 [00:42<06:18, 42.09s/it]

Mean: 0.5189743638038635  Median:  0.5189743638038635


 20%|██        | 2/10 [01:13<04:48, 36.10s/it]

Mean: 0.4933333396911621  Median:  0.4933333396911621


 30%|███       | 3/10 [01:49<04:11, 35.99s/it]

...

Mean: 0.4671794958412647  Median:  0.46948719024658203


 90%|█████████ | 9/10 [05:04<00:33, 33.03s/it]

Mean: 0.4665527145067851  Median:  0.4676923155784607


100%|██████████| 10/10 [05:37<00:00, 33.76s/it]

Mean: 0.4645641088485718  Median:  0.4646153897047043


In [33]:
model_scores_random

[0.519487202167511,
 0.47846153378486633,
 0.508717954158783,
 0.4558974504470825,
 0.45384615659713745]

In [35]:
np.mean(model_scores), np.mean(model_scores_random)

(0.5317948818206787, 0.48328205943107605)

In [34]:
np.mean(sorted(model_scores)[5:]), np.mean(sorted(model_scores_random)[5:])

/Users/dbugajny/PycharmProjects/CurriculumLearning/venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dbugajny/PycharmProjects/CurriculumLearning/venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


(nan, nan)

In [36]:
import scipy.stats
scipy.stats.ttest_ind(model_scores, model_scores_random)

TtestResult(statistic=3.476015034822166, pvalue=0.008368460131238117, df=8.0)